# Import packages and cleaned dataset

In [8]:
import pandas as pd
import numpy as np

In [9]:
supplyChain = pd.read_csv("data/DataCoSupplyChainDataset.csv", encoding='ISO-8859-1')

# Data Cleaning and Preprocessing

In [10]:
supplyChain_clean = supplyChain.drop(columns=['Days for shipping (real)', 'Delivery Status', 'Late_delivery_risk', 
                                        'shipping date (DateOrders)', 'Benefit per order', 'Sales per customer', 'Category Id',
                                        'Order Profit Per Order', 'Order Item Discount', 'Order Item Total', 'Order Status', 
                                        'Customer Email', 'Customer Password', 'Latitude', 'Longitude', 'Product Description', 'Product Image',
                                        'Customer Fname', 'Customer Id', 'Customer Lname', 'Department Id',
                                             'Order Customer Id', 'Order Item Cardprod Id', 'Order Item Id',
                                             'Product Card Id', 'Product Category Id', 'Order Id', 'Customer Street',
                                             'Customer Zipcode', 'Order Zipcode', 'Order Item Product Price',
                                             'Product Price', 'Order Item Profit Ratio', 'Product Status'])

In [11]:
supplyChain_clean.columns

Index(['Type', 'Days for shipment (scheduled)', 'Category Name',
       'Customer City', 'Customer Country', 'Customer Segment',
       'Customer State', 'Department Name', 'Market', 'Order City',
       'Order Country', 'order date (DateOrders)', 'Order Item Discount Rate',
       'Order Item Quantity', 'Sales', 'Order Region', 'Order State',
       'Product Name', 'Shipping Mode'],
      dtype='object')

In [12]:
categorical_cols = ['Type', 'Category Name', 'Customer City', 'Customer Country',
       'Customer Segment', 'Customer State', 'Department Name', 'Market',
       'Order City', 'Order Country',
       'Order Region', 'Order State', 'Product Name', 'Shipping Mode']

In [13]:
# Date and Time Features 
supplyChain_clean['order date (DateOrders)'] = pd.to_datetime(supplyChain_clean['order date (DateOrders)']) 
supplyChain_clean['Day of Week'] = supplyChain_clean['order date (DateOrders)'].dt.dayofweek 
supplyChain_clean['Month'] = supplyChain_clean['order date (DateOrders)'].dt.month 
supplyChain_clean['Year'] = supplyChain_clean['order date (DateOrders)'].dt.year 
supplyChain_clean['Week of Year'] = supplyChain_clean['order date (DateOrders)'].dt.isocalendar().week 

# Assuming supplyChain_clean is your DataFrame
supplyChain_clean['order date (DateOrders)'] = pd.to_datetime(supplyChain_clean['order date (DateOrders)'])

# Sorting the DataFrame by the 'order date (DateOrders)' column
supplyChain_clean.sort_values(by='order date (DateOrders)', inplace=True)

# Dropping the 'order date (DateOrders)' column
supplyChain_clean.drop(columns=['order date (DateOrders)'], inplace=True)

In [14]:
supplyChain_clean_2000 = supplyChain_clean.iloc[:2000]

In [15]:
supplyChain_clean_2000.to_csv("DataCo_cleaned.csv", index = False)

In [16]:
supplyChain_clean_2000

,Type,Days for shipment (scheduled),Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Name,Market,Order City,...,Order Item Quantity,Sales,Order Region,Order State,Product Name,Shipping Mode,Day of Week,Month,Year,Week of Year
33833,CASH,4,Camping & Hiking,Hickory,EE. UU.,Consumer,NC,Fan Shop,LATAM,Mexico City,...,1,299.980011,Central America,Distrito Federal,Diamondback Women's Serene Classic Comfort Bi,Standard Class,3,1,2015,1
77011,PAYMENT,4,Water Sports,Chicago,EE. UU.,Consumer,IL,Fan Shop,LATAM,Dos Quebradas,...,1,199.990005,South America,Risaralda,Pelican Sunstream 100 Kayak,Standard Class,3,1,2015,1
109322,PAYMENT,4,Women's Apparel,Chicago,EE. UU.,Consumer,IL,Golf,LATAM,Dos Quebradas,...,5,250.000000,South America,Risaralda,Nike Men's Dri-FIT Victory Golf Polo,Standard Class,3,1,2015,1
87884,PAYMENT,4,Men's Footwear,Chicago,EE. UU.,Consumer,IL,Apparel,LATAM,Dos Quebradas,...,1,129.990005,South America,Risaralda,Nike Men's CJ Elite 2 TD Football Cleat,Standard Class,3,1,2015,1
114915,CASH,4,Indoor/Outdoor Games,San Antonio,EE. UU.,Home Office,TX,Fan Shop,LATAM,Dos Quebradas,...,4,199.919998,South America,Risaralda,O'Brien Men's Neoprene Life Vest,Standard Class,3,1,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57298,DEBIT,4,Cleats,Caguas,Puerto Rico,Consumer,PR,Apparel,LATAM,Las Tunas,...,4,239.960007,Caribbean,Las Tunas,Perfect Fitness Perfect Rip Deck,Standard Class,0,1,2015,3
22197,DEBIT,4,Electronics,Caguas,Puerto Rico,Consumer,PR,Footwear,LATAM,Las Tunas,...,2,55.980000,Caribbean,Las Tunas,Under Armour Kids' Mercenary Slide,Standard Class,0,1,2015,3
20562,DEBIT,4,Camping & Hiking,Caguas,Puerto Rico,Consumer,PR,Fan Shop,LATAM,Las Tunas,...,1,299.980011,Caribbean,Las Tunas,Diamondback Women's Serene Classic Comfort Bi,Standard Class,0,1,2015,3
58105,DEBIT,4,Shop By Sport,Caguas,Puerto Rico,Consumer,PR,Golf,LATAM,Las Tunas,...,5,199.949997,Caribbean,Las Tunas,Under Armour Girls' Toddler Spine Surge Runni,Standard Class,0,1,2015,3
